### TEST DU CODE

In [2]:
import chess.svg
from anytree import NodeMixin
import numpy as np
import chess
from random import sample
from IPython.display import display

In [2]:
class node(NodeMixin):

    def __init__(self,move=None,parent=None):
        self.move = move 
        self.N = 0
        self.W = 0
        self.parent = parent

    def uct(self,white_to_play): # compute uct 
        if white_to_play:
            relative_W = self.W
        else:
            relative_W = -self.W
        #return relative_W/(self.N or 1) + 0.35*np.sqrt(np.log((self.parent.N or 1))/(self.N or 1))
        return 0.5*(relative_W*self.W + self.N)/(self.N or 1) + np.sqrt(2)*np.sqrt(np.log((self.parent.N or 1))/(self.N or 1)) # pour pouvoir utiliser sqrt(2) on se ramène dans [0,1]


In [3]:

class mcts():
    def __init__(self,position):
        self.initial_position = position.copy() # initial_position and current_position are chess.Board() objects 
        self.current_position = position.copy()
        self.root = node() # create the tree root, that correspond to initial_position

    def selection(self): # we reach a leaf using UCT selection
        current_node = self.root # we start from the root
        while current_node.is_leaf is not True: # check if we are in a leaf
            white_to_play = self.current_position.turn
            UCTs = [child.uct(white_to_play) for child in current_node.children] # list of UCTs for the node's children
            index = sample([i for i, j in enumerate(UCTs) if j == max(UCTs)],1)[0] # si plusieurs max tirage au hasard
            current_node = current_node.children[index] # the best children (with maximal UCT) becomes the current node
            self.current_position.push(current_node.move) # mise à jour de la position courante pour la phase d'expansion / simulation
        leaf = current_node # the current node is a leaf by definition
        return leaf

    def expansion_simulation(self,leaf): # we reach a final position starting from a previous leaf using random moves

        outcome = self.current_position.outcome() # résultat actuel de la partie
        #EXPANSION
        legal_moves = self.current_position.legal_moves # génération des coups légaux
        child_node = None # juste pour ne pas bugger la fin de partie

        if outcome is None: # si la partie n'est pas terminée
            chosen_move = sample(list(legal_moves),1)[0] # tirage aléatoire d'un coup légal
            for move in legal_moves: # création des nouveaux noeuds correspondants aux coups légaux
                if move == chosen_move:
                    child_node = node(move=move,parent=leaf) # On garde le noeud enfant dans une variable, on en aura besoin dans la rétropropagation
                else:
                    node(move=move,parent=leaf)
            self.current_position.push(chosen_move) # mise à jour de la position courante avec le nouveau coup
            outcome = self.current_position.outcome() # mise à jour de l'issue de la partie
        #SIMULATION
        while outcome is None : # tant que la partie n'est pas terminée
            legal_moves = self.current_position.legal_moves # génération des coups légaux
            random_move = sample(list(legal_moves),1)[0] # tirage aléatoire du prochain coup
            self.current_position.push(random_move) # mise à jour de la position courante avec le nouveau coup
            outcome = self.current_position.outcome() # mise à jour de l'issue de la partie
        result = outcome.result() 

        return child_node, result

    def backpropagation(self, child_node, result): # we backpropagate information through the tree
        if result == "1-0":
            inc = 1
        elif result == "0-1":
            inc = -1
        else:
            inc = 0
        if child_node is not None :  
            for ancestor in child_node.iter_path_reverse(): # rétropropagation du résultat issu de la simulation
                ancestor.N += 1
                ancestor.W += inc
        self.current_position = self.initial_position.copy()
        return


In [113]:
game = chess.Board()
MCTS = mcts(game)

while game.outcome() is None:

    for i in range(1000):
        leaf = MCTS.selection()
        child_node, result = MCTS.expansion_simulation(leaf)
        MCTS.backpropagation(child_node,result)
    
    N = [child.N for child in MCTS.root.children]
    print([[child.N, child.move] for child in MCTS.root.children])
    print(sum([child.N for child in MCTS.root.children]))
    index = sample([i for i, j in enumerate(N) if j == max(N)],1)[0] 
    move = [child.move for child in MCTS.root.children][index]
    game.push(move)

    # tree reuse
    MCTS.root = [child for child in MCTS.root.children][index]
    MCTS.root.parent = None
    MCTS.initial_position = game.copy()
    MCTS.current_position = game.copy()
    #MCTS = mcts(game)

    display(game)

'''CODER LA PARTIE CONTRE HUMAIN'''
display(game)  
    

KeyboardInterrupt: 

In [ ]:
def format_input_NN(chess_board):

    """
    :return: a representation of the board using an (18, 8, 8) shape, good as input to a policy / value network
    """

    pieces_order = 'KQRBNPkqrbnp' # 12x8x8
    castling_order = 'KQkq'       # 4x8x8
    # fifty-move-rule             # 1x8x8
    # en en_passant               # 1x8x8

    ind = {pieces_order[i]: i for i in range(12)}

    def canon_input_planes(fen):
        """

        :param fen:
        :return : (18, 8, 8) representation of the game state
        """
        fen = maybe_flip_fen(fen, is_black_turn(fen))
        return all_input_planes(fen)

    def all_input_planes(fen):
        current_aux_planes = aux_planes(fen)

        history_both = to_planes(fen)

        ret = np.vstack((history_both, current_aux_planes))
        assert ret.shape == (18, 8, 8)
        return ret

    def to_planes(fen):
        board_state = replace_tags_board(fen)
        pieces_both = np.zeros(shape=(12, 8, 8), dtype=np.float32)
        for rank in range(8):
            for file in range(8):
                v = board_state[rank * 8 + file]
                if v.isalpha():
                    pieces_both[ind[v]][rank][file] = 1
        assert pieces_both.shape == (12, 8, 8)
        return pieces_both

    def aux_planes(fen):
        foo = fen.split(' ')

        en_passant = np.zeros((8, 8), dtype=np.float32)
        if foo[3] != '-':
            eps = alg_to_coord(foo[3])
            en_passant[eps[0]][eps[1]] = 1

        fifty_move_count = int(foo[4])
        fifty_move = np.full((8, 8), fifty_move_count, dtype=np.float32)

        castling = foo[2]
        auxiliary_planes = [np.full((8, 8), int('K' in castling), dtype=np.float32),
                            np.full((8, 8), int('Q' in castling), dtype=np.float32),
                            np.full((8, 8), int('k' in castling), dtype=np.float32),
                            np.full((8, 8), int('q' in castling), dtype=np.float32),
                            fifty_move,
                            en_passant]

        ret = np.asarray(auxiliary_planes, dtype=np.float32)
        assert ret.shape == (6, 8, 8)
        return ret

    def replace_tags_board(board_san):
        board_san = board_san.split(" ")[0]
        board_san = board_san.replace("2", "11")
        board_san = board_san.replace("3", "111")
        board_san = board_san.replace("4", "1111")
        board_san = board_san.replace("5", "11111")
        board_san = board_san.replace("6", "111111")
        board_san = board_san.replace("7", "1111111")
        board_san = board_san.replace("8", "11111111")
        return board_san.replace("/", "")

    def is_black_turn(fen):
        return fen.split(" ")[1] == 'b'

    def alg_to_coord(alg):
        rank = 8 - int(alg[1])        # 0-7
        file = ord(alg[0]) - ord('a') # 0-7
        return rank, file

    def maybe_flip_fen(fen, flip = False):
        if not flip:
            return fen
        foo = fen.split(' ')
        rows = foo[0].split('/')
        def swapcase(a):
            if a.isalpha():
                return a.lower() if a.isupper() else a.upper()
            return a
        def swapall(aa):
            return "".join([swapcase(a) for a in aa])
        return "/".join([swapall(row) for row in reversed(rows)]) \
            + " " + ('w' if foo[1] == 'b' else 'b') \
            + " " + "".join(sorted(swapall(foo[2]))) \
            + " " + foo[3] + " " + foo[4] + " " + foo[5]

    return canon_input_planes(chess_board.fen())

### Nouveaux codes